### Milestone 1: collect, parse and organize firm 10-K forms (text).
Given a list of company names, download all 10-K filings. Parse 10-K filing and
only keep Item 1 (Business), item 1A (risk factors), item 7 (management’s
discussion and analysis of financial condition and results of operations. Text only).
Save data into database.

In [1]:
import edgar
from unidecode import unidecode
import pandas as pd

/home/jbai/anaconda3/envs/business/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/jbai/anaconda3/envs/business/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [8]:
def clear_file(doc):
    """
    @doc: the single doc got from  edgar.getDocuments
    @return: the cleared file, decode from unicode and remove white space
    """
    parsed_string = " ".join(str(doc).split())
    return unidecode(parsed_string)

def get10KByNameAndCIK(companyName, CIKNumber, noOfDocuments=1):
    """
    @companyName: the name of the company
    @CIKNumber: the cik number of the company
    @noOfDocuments: the number of differenct files for different years
    @return: the cleared file, decode from unicode and remove white space, in a list
    
    The user have to provide the CIK number by themselves
    """
    company = edgar.Company(companyName, CIKNumber)
    tree = company.getAllFilings(filingType = "10-K")
    docs = edgar.getDocuments(tree, noOfDocuments=noOfDocuments)
    if isinstance(docs, list):
        file_lists = [ clear_file(doc) for doc in docs]
        return file_lists
    else:
        return clear_file(docs)

def clear_cik_data(file_name):
    """
    @filename: "cik-lookup-data.txt"
    @return a table [[name, cik]...]
    """
    nameCIK_table = open(file_name, "rb")
    table = []
    for line in nameCIK_table:
        try:
            line = line.decode('utf8')
        except:
            print("Cannot decode utf-8: ", str(line))
            continue
        parts = line.split(":")
        parts.pop()
        cik = parts[-1]
        parts.pop()
        name = ":".join(parts)
        table.append((name, cik))
  
    df = pd.DataFrame(table)
    df.to_csv("company_cik.csv")
    return table

def getPartsFromParsedFile(file):
    return 0
    
def getRecentFileTable(company_list):
    """
    @company_list: the table of [[name, cik]...]
    @return table of 4 lists, name, cik, file content, file number
    
    for testing, set file number to be 3 and max number of company to be 20
    """
    n_file = 2
    max_n_company = 5
    
    comp_count = 0
    tenK_count = 0
    table = list()
    
    for company_pairs in company_list:
        # limit the amount of company, only for testing
        if tenK_count == max_n_company:
            break
            
        comp_count += 1
        files = get10KByNameAndCIK(company_pairs[0], company_pairs[1], n_file)
        if len(files) != n_file: 
            continue
        else:
            tenK_count += 1
            print(tenK_count)
            for i in range(len(files)):
                table.append((company_pairs[0], company_pairs[1],files[i], i))
        
    return table
            
        
    
    

In [9]:
company_list = clear_cik_data("cik-lookup-data.txt")

table = getRecentFileTable(company_list)


Cannot decode utf-8:  b'EXPLORA INVESTIMENTOS GEST\xc3O DE RECURSOS LTDA:0001538467:\n'
Cannot decode utf-8:  b'PRAGMA GEST\xc3O DE PATRIM\xd4NIO LTDA:0001555512:\n'
1
2
3
4
5


In [17]:
before_extract_df = pd.DataFrame(table)
before_extract_df[2]

'10-K 1 mktg_10k14.htm FORM 10-K UNITED STATES SECURITIES AND EXCHANGE COMMISSION Washington, D.C. 20549 FORM 10-K (Mark One) x ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the fiscal year ended March 31, 2014 or o TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the transition period from __________ to __________ Commission file number 0-20394 \'mktg, inc.\' (Exact name of registrant as specified in its charter) Delaware 06-1340408 (State or other jurisdiction of incorporation or organization) (I.R.S. Employer Identification No.) 75 Ninth Avenue, New York, New York 10011 (Address of principal executive offices) (Zip Code) Registrant\'s telephone number, including area code: (212) 366-3400 Securities registered pursuant to Section 12(b) of the Act: None Securities registered pursuant to Section 12(g) of the Act: Common Stock, $.001 par value per share Indicate by check mark if the registrant is a we

In [40]:
text = 'MSKSASPKEPEQLRKLFIGGLSFETTDESLRSAHFESSSYGSAGRRF'
  
search = 'SA'
start = 0
while True:
    index = text.find(search, start)
    # if search string not found, find() returns -1
    # search is complete, break out of the while loop
    if index == -1:
        break
    print( "%s found at index %d" % (search, index) )
    # move to next possible start position
    start = index + 1

SA found at index 3
SA found at index 31
SA found at index 41
